

# <center>Решающее дерево (Decision Tree)</center>

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

np.random.seed(42)

In [20]:
rdata = pd.read_csv("_ea07570741a3ec966e284208f588e50e_titanic.csv", index_col="PassengerId")
rdata = rdata.reindex(columns=["Sex", "Age", "Pclass", "Fare", "Survived"])

In [21]:
rdata.loc[rdata["Sex"] == "male", "Sex"] = 1
rdata.loc[rdata["Sex"] == "female", "Sex"] = 0
rdata = rdata.values

<b>Для начала организуем стандартную функцию разбиения на `train` и `test`</b>

In [4]:
# Первым аргументом она будет принимать датасет
# Вторым долю тестовой выборки от общей типа float, по умолчанию выставим 0.1

def train_test_split(data, ratio:float=0.1):
    
    # отберём индексы по количеству 
    ids = pd.DataFrame(data).index.tolist()
    
    # cгенерируем уникальные случайноые индексы из списка в заданном процентном соотношении
    ids = np.random.choice(ids, round(len(ids)*ratio), replace=False)
    
    # выберем строки с полученными индексами для тестовой выборки
    # исключим эти строки для тренировочной
    test = pd.DataFrame(data).loc[ids]
    train = pd.DataFrame(data).drop(ids)
    
    return np.array(train), np.array(test)

In [5]:
# Далее нам необходим функция, которая будет определять один ли класс остался
# в подвыборке или же нужно продолжать разбивать. Тут всё банально
# Если количество уникальных классов > 1 то возвращаем False
# Подразумевается, что принадлежность классу всегда в последней колонке

def is_one_class(data):
    if len(np.unique(data[:, -1])) == 1:
        return True
    else:
        return False

In [6]:
# Определим функцию листа дерева, которая будет классифицировать
# окончательно обработанные данные и возрщать их класс

# Так же стоит учесть заранее, что в данную подвыбоку могут попасть объекты разных классов,
# которые больше невозможно разделить по каким либо признакам
# Поэтому просто отнесём все объекты к самому распространённому классу в подвыборке
# при помощь функции argmax

def class_leaf(data):
    
    # В первой переменной будут находиться все уникальные классы
    # Во второй - как часто они они встречаются в подвыборке
    uniq_classes, uniq_classes_counts = np.unique(data[:, -1], return_counts=True)
    
    # Запомним индекс мажоритарного класса
    idx = np.argmax(uniq_classes_counts)
    
    # И вернём класс под этим индексом
    return uniq_classes[idx]

In [7]:
# Определим возможные пороги, по которым ещё можно разбить
# наши столбцы с признаками. Задача вернуть словарь вида:
# номер признака :  возможные пороги для разбиения

def get_thresholds(data):
    thresholds = {}
    
    # Определим количесво фичей (столбцов)
    n_features = data.shape[1]
    
    # И для каждой фичи (исключая таргет)
    # Определим все уникальные значения
    for n_feature in range(n_features - 1):
        thresholds[n_feature] = []
        unique_values = np.unique(data[:, n_feature])
        
        # Теперь по всему столбцу признаков определим порог
        # (среднее), по которому потенциально можно будет разделить выборку
        for idx in range(len(unique_values)):
            if idx:
                cur_value = unique_values[idx]
                prev_value = unique_values[idx - 1]
                possible_split = (cur_value + prev_value) / 2
                thresholds[n_feature].append(possible_split)
    return thresholds

In [8]:
# Теперь нам понадобится функция разбиения подвыборки на ветви
# по данному порогу и столбцу

def split_by_class(data, column, thresh):
    
    left_branch = data[data[:, column] <= thresh]
    right_branch = data[data[:, column] > thresh]
    
    return left_branch, right_branch

 -----------
     Для определения наиболее оптимального порога для разбиения подвыборки используется несколько критериев информативности.
    Рассмотрим наиболее часто используемые:
    
    Подробнее о них:
   [КРИТЕРИЙ ИНФОРМАТИВНОСТИ](http://www.machinelearning.ru/wiki/images/8/89/Sem3_trees.pdf)

------------

## Энтропийный критерий

    В итоге нам нужно найти минимальную энтропию при всех возможных распределениях подвыборки по ветвям 
    (по всем столбцам и порогам)

--------------

#### Энтропия по подвыборке
## $$ H(p) = \sum_{i=1}^cp_i  (-log_2 p_i)$$

--------------

#### Энтропийный критерий информативности

## $$ Q_H = \frac{N_{l}}{N_m}H(p_{l}) +  \frac{N_{r}}{N_m}H(p_{r}) \rightarrow min$$

Где <b>_l_</b> - подвыборка из левой ветви, а <b>_r_</b> - подвыборка из правой ветви

-------------


In [9]:
# Посчитаем энтропию в подвыборке

def get_entropy(data):
    
    # для начала, определим долю каждого класса в выборке
    # нам понадобится общее количесво объектов каждого класса в ней
    _, uniq_classes_counts = np.unique(data[:, -1], return_counts=True)
    
    # разделим количество каждого класса на общее их число и получим долю каждого класса
    rate = uniq_classes_counts / uniq_classes_counts.sum()
    
    # теперь по формуле может посчитать энтропию в этой подвыборке
    entropy = sum(rate * -np.log2(rate))
    
    return entropy

In [ ]:
def gini(data):
    _, uniq_classes_counts = np.unique(data[:, -1], return_counts=True)
    rate = uniq_classes_counts / uniq_classes_counts.sum()
    gini = 1 - sum(rate ** 2)

In [10]:
# Теперь можно задать функцию подсчёта энтропии по двум ветвям при определённом распределении

def get_overall_entropy(left_branch, right_branch):
    
    n_all = len(left_branch) + len(right_branch)
    n_left = len(left_branch) / n_all
    n_right = len(right_branch) / n_all
    
    # так же по следующей формуле считаем суммарную энтропию на общую долю
    overall_entropy = (n_left * get_entropy(left_branch) + n_right * get_entropy(right_branch))
    
    return overall_entropy

In [11]:
# И так, у нас всё готово для поиска критерия информативности
# через подбор оптимальной (минимальной) энтропии

def impurity_function(data, thresholds):
    
    overall_entropy = 666
    
    # для каждого стоблца в словаре со всеми возможными порогами
    for col in thresholds:
        # для каждого порога в нём
        for thresh in thresholds[col]:
            # разобьём подвыборку на ветви и подсчтаем энтропию для каждого разбиения
            left_branch, right_branch = split_by_class(data, column=col, thresh=thresh)
            i_entropy = get_overall_entropy(left_branch, right_branch)
            
            # и если текущее значение энтропии меньше имеющегося, то обновляем значения
            if i_entropy <= overall_entropy:
                overall_entropy = i_entropy
                optimal_col = col
                optimal_thresh = thresh
    
    return optimal_col, optimal_thresh

In [12]:
# Пришло время обернуть всё в рекурсивный алгоримт дерева,
# который будет иметь вид словаря из словарей, представляющего наше дерево

def tree(data, counter=0, min_samples=2, max_depth=5):
    
    # крайний случай (если находимся в терминальной ветке)
    # тогда вернём предсказание класса в этой ветке
    if is_one_class(data) or len(data) < min_samples or counter > max_depth:
        return class_leaf(data)
    
    # иначе рекурсия ...
    else:
        counter += 1
    
    # разобьём подвыборку на ветви по критерию информативности
    thresholds = get_thresholds(data)
    
    # если нет порогов для разбиения классов - предскажем класс
    is_thresholds = 0
    for i in thresholds:
        is_thresholds += len(thresholds[i])
    if not is_thresholds:
        return class_leaf(data)
    
    optimal_column, optimal_threshold = impurity_function(data, thresholds)
    left_branch, right_branch = split_by_class(data, optimal_column, optimal_threshold)
    
    # определим, как будет выглядеть наш словарь с вопросами-ответами
    # обозначим условие разбиения
    condition = "{} <= {}".format(optimal_column, optimal_threshold)
    sub_tree = {condition: []}
    
    # разбиваем на ветви дальше пока не классифицируем все объекты
    to_left = tree(left_branch, counter, min_samples, max_depth)
    to_right = tree(right_branch, counter, min_samples, max_depth)
    
    if to_left == to_right:
        sub_tree = to_left
    else:
        sub_tree[condition].append(to_left)
        sub_tree[condition].append(to_right)
    
    return sub_tree

In [15]:
derevo = tree(rdata)

In [16]:
derevo

{'0 <= 0.5': [{'2 <= 2.5': [{'3 <= 28.85625': [{'3 <= 28.23125': [1, 0]},
      {'1 <= 2.5': [0,
        {'3 <= 149.0354': [1, {'3 <= 152.50625000000002': [0, 1]}]}]}]},
    {'3 <= 20.799999999999997': [{'1 <= 16.5': [1,
        {'1 <= 36.5': [{'3 <= 18.62915': [0, 1]}, {'1 <= 55.0': [0, 1]}]}]},
      {'1 <= 5.5': [{'1 <= 3.5': [0, 1]},
        {'3 <= 31.331249999999997': [0, {'3 <= 32.88125': [1, 0]}]}]}]}]},
  {'2 <= 1.5': [{'1 <= 53.0': [{'3 <= 25.9375': [0,
        {'3 <= 27.1354': [1, {'1 <= 17.5': [1, 0]}]}]},
      {'1 <= 75.5': [{'3 <= 35.0771': [0, {'3 <= 42.5021': [1, 0]}]}, 1]}]},
    {'1 <= 9.5': [{'3 <= 20.825': [1, {'2 <= 2.5': [1, 0]}]}, 0]}]}]}

In [ ]:
f_name, _, val = list(derevo.keys())[0].split()

In [ ]:
ldata = pd.read_csv("_ea07570741a3ec966e284208f588e50e_titanic.csv", index_col="PassengerId")
ldata = ldata.reindex(columns=["Sex", "Age", "Pclass", "Fare", "Survived"]).dropna()
ldata.loc[ldata["Sex"] == "male", "Sex"] = 1
ldata.loc[ldata["Sex"] == "female", "Sex"] = 0
ldata = ldata.values

In [ ]:
ldata[2]

In [17]:
# Теперь на базе сформированного дерева можем делать предсказание

def predict(obj, tree):
    
    # представим первое условие в поддереве в виде списка
    # и разобьём на индекс признака и значение порога
    condition = list(tree.keys())[0]
    f_name, _, val = condition.split()
    
    # если условие выполняется то ответ в левой части дерева иначе в правой
    if obj[int(f_name)] <= float(val):
        answer = tree[condition][0]
    else:
        answer = tree[condition][1]
        
    # если ответом является классификация в терминальной ветке, а не словарь, то возвращаем его
    # иначе отправляемся дальше по словарю
    if not isinstance(answer, dict):
        return answer
    else:
        return predict(obj, answer)

In [18]:
def accuracy(data, tree):
    data = pd.DataFrame(data)
    target = data.iloc[:, -1]
    data['classify'] = data.apply(predict, axis=1, args=(tree,))
    data['correct'] = data.classify == target
    return data.correct.mean()

In [22]:
accuracy(rdata, derevo)

0.8451178451178452